In [1]:
import pulp
import pandas
import numpy as np

In [2]:
Calidad={"Primera","Segunda","Tercera"}
TiposAditivos={"A","B"}
         
Stock={"A":200,"B":1000}
MBase=1500
Req={"Primera":{"A":0.500,"B":0},
     "Segunda":{"A":0.250,"B":0.350},
     "Tercera":{"A":0,"B":0.800}}
Precio={"Primera":2000,"Segunda":1800,"Tercera":1500}


In [3]:
prob = pulp.LpProblem("Fabrica_ladrillos", pulp.LpMaximize)

x = pulp.LpVariable.dicts("Produccion", (c for c in Calidad), lowBound=0, cat='Continuous')


In [4]:
prob += pulp.lpSum([x[c]*Precio[c] for c in Calidad])


In [5]:
for i in TiposAditivos:
    prob += 0<=pulp.lpSum([x[c]*Req[c][i] for c in Calidad])<= Stock[i]

prob += pulp.lpSum([x[c] for c in Calidad]) <= MBase

# for i in TiposAditivos:
#     prob += pulp.lpSum([x[c]*Req[c][i] for c in Calidad])== a[i]

# prob += pulp.lpSum([x[c] for c in Calidad]) <= MBase

# for i in TiposAditivos :
#     prob += a[i]<= Stock[i]


In [6]:
# Choose solver, and set it to problem, and build the Gurobi model
solver = pulp.GUROBI()
prob.setSolver(solver)
prob.solver.buildSolverModel(prob)

# Solve model
prob.solve()

print("#################################################################")

print("\nVariables:\n")

for i in prob.variables():
    print(i, i.varValue)


Using license file /home/maxi/anaconda3/gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (linux64)
Optimize a model with 3 rows, 3 columns and 7 nonzeros
Model fingerprint: 0x915cddde
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [2e+03, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 2e+03]
Presolve time: 0.02s
Presolved: 3 rows, 3 columns, 7 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.3000000e+33   5.650000e+30   5.300000e+03      0s
       3    2.4900000e+06   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds
Optimal objective  2.490000000e+06
Gurobi status= 2
#################################################################

Variables:

Produccion_Primera 0.0
Produccion_Segunda 800.0
Produccion_Tercera 700.0


In [7]:
prob

Fabrica_ladrillos:
MAXIMIZE
2000*Produccion_Primera + 1800*Produccion_Segunda + 1500*Produccion_Tercera + 0
SUBJECT TO
_C1: 0.5 Produccion_Primera + 0.25 Produccion_Segunda <= 200

_C2: 0.35 Produccion_Segunda + 0.8 Produccion_Tercera <= 1000

_C3: Produccion_Primera + Produccion_Segunda + Produccion_Tercera <= 1500

VARIABLES
Produccion_Primera Continuous
Produccion_Segunda Continuous
Produccion_Tercera Continuous

In [8]:
sol=[x["Primera"].varValue,x["Segunda"].varValue,x["Tercera"].varValue]

In [9]:
import numpy as np
from sympy import Plane, Point3D
import networkx as nx


def simplify(triangles):
    """
    Simplify an iterable of triangles such that adjacent and coplanar triangles form a single face.
    Each triangle is a set of 3 points in 3D space.
    """

    # create a graph in which nodes represent triangles;
    # nodes are connected if the corresponding triangles are adjacent and coplanar
    G = nx.Graph()
    G.add_nodes_from(range(len(triangles)))
    for ii, a in enumerate(triangles):
        for jj, b in enumerate(triangles):
            if (ii < jj): # test relationships only in one way as adjacency and co-planarity are bijective
                if is_adjacent(a, b):
                    if is_coplanar(a, b, np.pi / 180.):
                        G.add_edge(ii,jj)

    # triangles that belong to a connected component can be combined
    components = list(nx.connected_components(G))
    simplified = [set(flatten(triangles[index] for index in component)) for component in components]

    # need to reorder nodes so that patches are plotted correctly
    reordered = [reorder(face) for face in simplified]

    return reordered


def is_adjacent(a, b):
    return len(set(a) & set(b)) == 2 # i.e. triangles share 2 points and hence a side


def is_coplanar(a, b, tolerance_in_radians=0):
    a1, a2, a3 = a
    b1, b2, b3 = b
    plane_a = Plane(Point3D(a1), Point3D(a2), Point3D(a3))
    plane_b = Plane(Point3D(b1), Point3D(b2), Point3D(b3))
    if not tolerance_in_radians: # only accept exact results
        return plane_a.is_coplanar(plane_b)
    else:
        angle = plane_a.angle_between(plane_b).evalf()
        angle %= np.pi # make sure that angle is between 0 and np.pi
        return (angle - tolerance_in_radians <= 0.) or \
            ((np.pi - angle) - tolerance_in_radians <= 0.)


flatten = lambda l: [item for sublist in l for item in sublist]


def reorder(vertices):
    """
    Reorder nodes such that the resulting path corresponds to the "hull" of the set of points.

    Note:
    -----
    Not tested on edge cases, and likely to break.
    Probably only works for convex shapes.

    """
    if len(vertices) <= 3: # just a triangle
        return vertices
    else:
        # take random vertex (here simply the first)
        reordered = [vertices.pop()]
        # get next closest vertex that is not yet reordered
        # repeat until only one vertex remains in original list
        vertices = list(vertices)
        while len(vertices) > 1:
            idx = np.argmin(get_distance(reordered[-1], vertices))
            v = vertices.pop(idx)
            reordered.append(v)
        # add remaining vertex to output
        reordered += vertices
        return reordered


def get_distance(v1, v2):
    v2 = np.array(list(v2))
    difference = v2 - v1
    ssd = np.sum(difference**2, axis=1)
    return np.sqrt(ssd)

In [10]:
%matplotlib widget

from scipy.spatial import HalfspaceIntersection
from scipy.spatial import ConvexHull
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as a3
import matplotlib.colors as colors


w = np.array([1., 1., 1.])


# ∑ᵢ hᵢ wᵢ qᵢ - ∑ᵢ gᵢ wᵢ <= 0
#  qᵢ - ubᵢ <= 0
# -qᵢ + lbᵢ <= 0
halfspaces = np.array([
                    [1.*w[0], 1.*w[1], 1.*w[2], -1500 ],
                    [ 0,  0.350,  0.800, -1000],
                    [ 0.500,  0.250,  0., -200],
                    [-1.,  0.,  0.,  0],
                    [ 0., -1.,  0.,  0],
                    [ 0.,  0., -1.,  0]
                    ])
feasible_point = np.array([0.1, 0.1, 0.11])
hs = HalfspaceIntersection(halfspaces, feasible_point)
verts = hs.intersections
verts = np.around(verts)
hull = ConvexHull(verts)
faces = hull.simplices

fig=plt.figure()
fig.canvas.layout.width = '7in'

ax = a3.Axes3D(fig)
ax.dist=10
ax.azim=30
ax.elev=10

lim_sup=np.max(verts)
lim_inf=np.min(verts)
ax.set_xlim([lim_inf,lim_sup])
ax.set_ylim([lim_inf,lim_sup])
ax.set_zlim([lim_inf,lim_sup])



triangles = []
for s in faces:
    sq = [
        (verts[s[0], 0], verts[s[0], 1], verts[s[0], 2]),
        (verts[s[1], 0], verts[s[1], 1], verts[s[1], 2]),
        (verts[s[2], 0], verts[s[2], 1], verts[s[2], 2])
    ]
    triangles.append(sq)

new_faces = simplify(triangles)
for sq in new_faces:
    f = a3.art3d.Poly3DCollection([sq])
    f.set_color(colors.rgb2hex(sp.rand(3)))
    f.set_edgecolor('k')
    f.set_alpha(0.1)
    ax.add_collection3d(f)

    
    
ax.scatter(sol[0],sol[1],sol[2],s=100)
ax.set_xlabel('Primera')
ax.set_ylabel('Segunda')
ax.set_zlabel('Tercera')

# # rotate the axes and update
# for angle in range(0, 360):
#     ax.view_init(30, angle)
#     plt.draw()
#     plt.pause(.001)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/maxi/anaconda3/envs/opti/lib/python3.6/site-packages/ipykernel_launcher.py:61: DeprecationWarning: scipy.rand is deprecated and will be removed in SciPy 2.0.0, use numpy.random.rand instead


Text(0.5, 0, 'Tercera')

In [11]:
s


array([5, 7, 6], dtype=int32)